In [1]:
import subprocess
import shlex
import json
from glob import glob
from tqdm import tqdm_notebook as tqdm
import pandas as pd

import sys
import os
sys.path.append(os.path.join(sys.path[0], '../'))

from core.train_utils import calculate_roi, get_winner_favorite
from core.events_utils import parse_odds

In [2]:
df = pd.read_csv('./v2_1/data_models/events_joined_total_11.05.2021.csv', index_col=0)
cat_cols = ['city', 'country', 'is_fight_night', 'weightCategory.id', 
              'f1_country', 'f2_country', 'f1_city', 'f2_city']

df[cat_cols] = df[cat_cols].fillna('unknown')
# df = df[(df['f1_count_of_fights'] >= 1) & (df['f2_count_of_fights'] >= 1)]

df['eventDate.date'] = pd.to_datetime(df['eventDate.date'])
df['weightCategory.id'] = df['weightCategory.id'].astype('object')

df  = df[(df['eventDate.date'].dt.year == 2021) &
        ((df['type']=='straight'))]
df = df[~df['odds_difference'].isna()]

df

,city,country,duration,eventDate.date,eventDate.timezone,fighterId_1,fighterId_2,id,name,rounds,...,age_difference,height_difference,weight_difference,armSpan_difference,legSwing_difference,isHomeCity_difference,isHomeCountry_difference,isHomeTimezone_difference,winner,is_fight_night
5791,Abu Dhabi,United Arab Emirates,300.0,2021-01-16 00:00:01,Europe/Berlin,2597,3418,139,UFC Fight Night,3.0,...,-1.0,-2.54,0.00,5.08,NaN,0.0,0.0,0.0,False,True
5792,Abu Dhabi,United Arab Emirates,300.0,2021-01-16 00:00:02,Europe/Berlin,3398,240,138,UFC Fight Night,3.0,...,0.0,-5.08,0.00,-5.08,NaN,0.0,0.0,0.0,True,True
5793,Abu Dhabi,United Arab Emirates,300.0,2021-01-16 00:00:03,Europe/Berlin,141,453,136,UFC Fight Night,3.0,...,-3.0,5.08,0.00,-2.54,NaN,0.0,0.0,0.0,False,True
5794,Abu Dhabi,United Arab Emirates,300.0,2021-01-16 00:00:04,Europe/Berlin,3219,2319,137,UFC Fight Night,3.0,...,1.0,0.00,8.62,-2.54,NaN,0.0,0.0,0.0,False,True
5795,Abu Dhabi,United Arab Emirates,300.0,2021-01-16 00:00:05,Europe/Berlin,3666,462,135,UFC Fight Night,3.0,...,1.0,-5.08,4.54,0.00,NaN,0.0,0.0,0.0,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5920,Las Vegas,USA,82.0,2021-04-10 00:00:07,Europe/Berlin,3587,430,262,UFC Fight Night,2.0,...,-7.0,2.54,0.00,2.54,NaN,0.0,-1.0,0.0,True,True
5922,Las Vegas,USA,300.0,2021-04-10 00:00:09,Europe/Berlin,3037,288,261,UFC Fight Night,3.0,...,5.0,7.62,0.00,7.62,NaN,0.0,0.0,0.0,True,True
5923,Las Vegas,USA,288.0,2021-04-10 00:00:10,Europe/Berlin,600,552,258,UFC Fight Night,1.0,...,-7.0,-2.54,0.00,-2.54,-5.08,0.0,0.0,0.0,True,True
5924,Las Vegas,USA,127.0,2021-04-10 00:00:11,Europe/Berlin,449,215,259,UFC Fight Night,2.0,...,4.0,0.00,9.08,7.62,2.54,0.0,0.0,0.0,False,True


In [13]:
events_df = pd.read_csv('../data/0.events_raw.csv', index_col=0)
### Убираем строки, где `winnerId` не совпадает с айди ни одного из бойцов
events_df = events_df[~((events_df["winnerId"] != events_df["fighterId_1"]) & (events_df["winnerId"] != events_df["fighterId_2"]))]

events_df[["f1_odds", "f2_odds"]] = events_df[["avgOdds", "fighterId_1", "fighterId_2"]]\
            .apply(lambda row: parse_odds(row), axis=1)

events_df = events_df[~events_df['f1_odds'].isna()]
events_df = events_df.reset_index()

events_df['eventDate.date'] = pd.to_datetime(events_df['eventDate.date'])
events_df = events_df[(events_df['eventDate.date'].dt.year == 2021)]

events_df['f1_count_of_fights'] = df['f1_count_of_fights'].values
events_df['f2_count_of_fights'] = df['f2_count_of_fights'].values
events_df

,index,avgOdds,city,completed,country,duration,eventDate.date,eventDate.timezone,eventDate.timezone_type,fighterId_1,...,rounds,timezone,weightCategory.id,weightCategory.name,winMethods,winnerId,f1_odds,f2_odds,f1_count_of_fights,f2_count_of_fights
542,0,"[{'fighterId': 2597, 'value': 2.95}, {'fighter...",Abu Dhabi,True,United Arab Emirates,300.0,2021-01-16 00:00:01,Europe/Berlin,3,2597,...,3.0,Asia/Dubai,4,Полулегкий вес,['DEC'],3418.0,2.95,1.43,1,1
543,0,"[{'fighterId': 240, 'value': 1.5}, {'fighterId...",Abu Dhabi,True,United Arab Emirates,300.0,2021-01-16 00:00:02,Europe/Berlin,3,240,...,3.0,Asia/Dubai,12,Женский легчайший вес,['DEC'],3398.0,1.50,2.70,3,8
544,0,"[{'fighterId': 141, 'value': 3.37}, {'fighterI...",Abu Dhabi,True,United Arab Emirates,300.0,2021-01-16 00:00:03,Europe/Berlin,3,141,...,3.0,Asia/Dubai,6,Полусредний вес,['DEC'],453.0,3.37,1.34,3,5
545,0,"[{'fighterId': 2319, 'value': 1.49}, {'fighter...",Abu Dhabi,True,United Arab Emirates,300.0,2021-01-16 00:00:04,Europe/Berlin,3,2319,...,3.0,Asia/Dubai,9,Тяжелый вес,['DEC'],2319.0,1.49,2.72,2,2
546,0,"[{'fighterId': 462, 'value': 1.93}, {'fighterI...",Abu Dhabi,True,United Arab Emirates,300.0,2021-01-16 00:00:05,Europe/Berlin,3,462,...,3.0,Asia/Dubai,12,Женский легчайший вес,['DEC'],3666.0,1.93,1.93,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659,0,"[{'fighterId': 430, 'value': 3}, {'fighterId':...",Las Vegas,True,USA,82.0,2021-04-10 00:00:07,Europe/Berlin,3,430,...,2.0,America/Los_Angeles,5,Легкий вес,['KO'],3587.0,3.00,1.42,1,11
660,0,"[{'fighterId': 288, 'value': 2.22}, {'fighterI...",Las Vegas,True,USA,300.0,2021-04-10 00:00:09,Europe/Berlin,3,288,...,3.0,America/Los_Angeles,6,Полусредний вес,['DEC'],3037.0,2.22,1.71,4,14
661,0,"[{'fighterId': 552, 'value': 1.74}, {'fighterI...",Las Vegas,True,USA,288.0,2021-04-10 00:00:10,Europe/Berlin,3,552,...,1.0,America/Los_Angeles,10,Женский минимальный вес,['SUB'],600.0,1.74,2.17,6,7
662,0,"[{'fighterId': 215, 'value': 1.54}, {'fighterI...",Las Vegas,True,USA,127.0,2021-04-10 00:00:11,Europe/Berlin,3,215,...,2.0,America/Los_Angeles,7,Средний вес,['SUB'],215.0,1.54,2.58,19,3


In [19]:
df

,city,country,duration,eventDate.date,eventDate.timezone,fighterId_1,fighterId_2,id,name,rounds,...,age_difference,height_difference,weight_difference,armSpan_difference,legSwing_difference,isHomeCity_difference,isHomeCountry_difference,isHomeTimezone_difference,winner,is_fight_night
5791,Abu Dhabi,United Arab Emirates,300.0,2021-01-16 00:00:01,Europe/Berlin,2597,3418,139,UFC Fight Night,3.0,...,-1.0,-2.54,0.00,5.08,NaN,0.0,0.0,0.0,False,True
5792,Abu Dhabi,United Arab Emirates,300.0,2021-01-16 00:00:02,Europe/Berlin,3398,240,138,UFC Fight Night,3.0,...,0.0,-5.08,0.00,-5.08,NaN,0.0,0.0,0.0,True,True
5793,Abu Dhabi,United Arab Emirates,300.0,2021-01-16 00:00:03,Europe/Berlin,141,453,136,UFC Fight Night,3.0,...,-3.0,5.08,0.00,-2.54,NaN,0.0,0.0,0.0,False,True
5794,Abu Dhabi,United Arab Emirates,300.0,2021-01-16 00:00:04,Europe/Berlin,3219,2319,137,UFC Fight Night,3.0,...,1.0,0.00,8.62,-2.54,NaN,0.0,0.0,0.0,False,True
5795,Abu Dhabi,United Arab Emirates,300.0,2021-01-16 00:00:05,Europe/Berlin,3666,462,135,UFC Fight Night,3.0,...,1.0,-5.08,4.54,0.00,NaN,0.0,0.0,0.0,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5920,Las Vegas,USA,82.0,2021-04-10 00:00:07,Europe/Berlin,3587,430,262,UFC Fight Night,2.0,...,-7.0,2.54,0.00,2.54,NaN,0.0,-1.0,0.0,True,True
5922,Las Vegas,USA,300.0,2021-04-10 00:00:09,Europe/Berlin,3037,288,261,UFC Fight Night,3.0,...,5.0,7.62,0.00,7.62,NaN,0.0,0.0,0.0,True,True
5923,Las Vegas,USA,288.0,2021-04-10 00:00:10,Europe/Berlin,600,552,258,UFC Fight Night,1.0,...,-7.0,-2.54,0.00,-2.54,-5.08,0.0,0.0,0.0,True,True
5924,Las Vegas,USA,127.0,2021-04-10 00:00:11,Europe/Berlin,449,215,259,UFC Fight Night,2.0,...,4.0,0.00,9.08,7.62,2.54,0.0,0.0,0.0,False,True


In [20]:
events_df[events_df['f1_count_of_fights'] == 0][['fighterId_1', 'fighterId_2', 'eventDate.date', ]]

,fighterId_1,fighterId_2,eventDate.date
546,462,3666,2021-01-16 00:00:05
552,3610,3664,2021-01-20 00:00:01
553,3411,3428,2021-01-20 00:00:02
554,514,3599,2021-01-20 00:00:03
555,3482,3622,2021-01-20 00:00:04
558,353,3667,2021-01-20 00:00:07
573,434,2154,2021-01-23 00:00:10
581,476,3435,2021-02-06 00:00:09
585,3053,3442,2021-02-13 00:00:01
611,156,3579,2021-02-27 00:00:05


In [4]:
indexes = events_df.index
f1_id, f2_id, f1_odd, f2_odd, weightCategory_id, city, country, eventDate, eventName, eventTimezone = \
    events_df.loc[indexes[100], ['fighterId_1', 'fighterId_2', 'f1_odds', 'f2_odds', 'weightCategory.id',
                        'city', 'country', 'eventDate.date', 'name', 'timezone']]


f1_id, f2_id, f1_odd, f2_odd, weightCategory_id, city, country, eventDate, eventName, eventTimezone


(3447,
 3520,
 2.9,
 1.44,
 3,
 'Las Vegas',
 'USA',
 Timestamp('2021-03-20 00:00:07'),
 'UFC Fight Night',
 'America/Los_Angeles')

In [24]:
import datetime
f1_id, f2_id = 3610, 3664
eventDate = datetime.date(2021, 1, 20)
eventTimezone = 'Asia/Dubai'
Curl = f"""curl -X 'GET' \
  'http://127.0.0.1:8001/predict_fight?f1_id={f1_id}&f2_id={f2_id}&f1_odd={f1_odd}&f2_odd={f2_odd}&\
weightCategory_id={weightCategory_id}&\
city={city.replace(' ', '%20')}&\
country={country.replace(' ', '%20')}&\
event_date={str(eventDate)[:10]}&\
event_name={eventName.replace(' ', '%20')}&\
time_zone={eventTimezone}' \
  -H 'accept: application/json'"""

print(Curl)
args = shlex.split(Curl)
process = subprocess.Popen(args, shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
response = json.loads(stdout.decode("UTF-8"))
# v0_0 = response['y_proba_catboost_v1_0']
# v0_1 = response['y_proba_catboost_v1_1']
# v0_2 = response['y_proba_catboost_v1_2']
response

curl -X 'GET'   'http://127.0.0.1:8001/predict_fight?f1_id=3610&f2_id=3664&f1_odd=2.25&f2_odd=1.69&weightCategory_id=4&city=Las%20Vegas&country=USA&event_date=2021-01-20&event_name=UFC%20Fight%20Night&time_zone=Asia/Dubai'   -H 'accept: application/json'


{'success': False, 'exp': {}}

In [8]:
for ind in tqdm(events_df.index[:]):
    f1_id, f2_id, f1_odd, f2_odd, weightCategory_id, city, country, eventDate, eventName, eventTimezone = \
        events_df.loc[ind, ['fighterId_1', 'fighterId_2', 'f1_odds', 'f2_odds', 'weightCategory.id',
                            'city', 'country', 'eventDate.date', 'name', 'timezone']]

    f1_id, f2_id, f1_odd, f2_odd, weightCategory_id, city, country, eventDate, eventName, eventTimezone
   
    Curl = f"""curl -X 'GET' \
  'http://127.0.0.1:8001/predict_fight?f1_id={f1_id}&f2_id={f2_id}&f1_odd={f1_odd}&f2_odd={f2_odd}&\
weightCategory_id={weightCategory_id}&\
city={city.replace(' ', '%20')}&\
country={country.replace(' ', '%20')}&\
event_date={str(eventDate)[:10]}&\
event_name={eventName.replace(' ', '%20')}&\
time_zone={eventTimezone}' \
  -H 'accept: application/json'"""

    args = shlex.split(Curl)
    process = subprocess.Popen(args, shell=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    response = json.loads(stdout.decode("UTF-8"))
    if response['success']:    

        events_df.at[ind, 'v1_0'] = response['y_proba_catboost_v1_0']
        events_df.at[ind, 'v1_1'] = response['y_proba_catboost_v1_1']
        events_df.at[ind, 'v1_2'] = response['y_proba_catboost_v1_2']
        events_df.at[ind, 'v1_1_2020'] = response['y_proba_catboost_v1_1_2020']
        events_df.at[ind, 'v1_2_2020'] = response['y_proba_catboost_v1_2_2020']
#         events_df.at[ind, 'v1_1_copy1'] = response['y_proba_catboost_v1_1_copy1']
        events_df.at[ind, 'v2_1'] = response['y_proba_catboost_v2_1']


/home/nikolay/anaconda3/envs/must/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/122 [00:00<?, ?it/s]

In [11]:
events_df['winner'] = events_df['winnerId'] == events_df['fighterId_1']

events_df['v1_0_pred'] = events_df['v1_0'] > 0.5
events_df['v1_1_pred'] = events_df['v1_1'] > 0.5
events_df['v1_2_pred'] = events_df['v1_2'] > 0.5
events_df['v1_1_2020_pred'] = events_df['v1_1_2020'] > 0.5
events_df['v1_2_2020_pred'] = events_df['v1_2_2020'] > 0.5
# events_df['v1_1_copy1_pred'] = events_df['v1_1_copy1'] > 0.5
events_df['v2_1_pred'] = events_df['v2_1'] > 0.5

        
print('v1_0_pred', (events_df['v1_0_pred'] == events_df['winner']).mean())
print('v1_1_pred', (events_df['v1_1_pred'] == events_df['winner']).mean())
print('v1_2_pred', (events_df['v1_2_pred'] == events_df['winner']).mean())
print('v1_1_2020_pred', (events_df['v1_1_2020_pred'] == events_df['winner']).mean())
print('v1_2_2020_pred', (events_df['v1_2_2020_pred'] == events_df['winner']).mean())
# print('v1_1_copy1', (events_df['v1_1_copy1_pred'] == events_df['winner']).mean())
print('v2_1', (events_df['v2_1_pred'] == events_df['winner']).mean())

v1_0_pred 0.5491803278688525
v1_1_pred 0.6311475409836066
v1_2_pred 0.5901639344262295
v1_1_2020_pred 0.6065573770491803
v1_2_2020_pred 0.5737704918032787
v2_1 0.639344262295082


In [51]:
q = events_df[(events_df['f1_count_of_fights'] >= 1) & (events_df['f2_count_of_fights'] >= 1)]
      
print('v1_0_pred', (q['v1_0_pred'] == q['winner']).mean())
print('v1_1_pred', (q['v1_1_pred'] == q['winner']).mean())
print('v1_2_pred', (q['v1_2_pred'] == q['winner']).mean())
print('v1_1_2020_pred', (q['v1_1_2020_pred'] == q['winner']).mean())
print('v1_2_2020_pred', (q['v1_2_2020_pred'] == q['winner']).mean())
# print('v1_1_copy1_pred', (q['v1_1_copy1_pred'] == q['winner']).mean())
print('v2_1_copy1_pred', (q['v2_1_pred'] == q['winner']).mean())

v1_0_pred 0.56
v1_1_pred 0.66
v1_2_pred 0.61
v1_1_2020_pred 0.63
v1_2_2020_pred 0.59
v1_1_copy1_pred 0.6
v2_1_copy1_pred 0.67


In [57]:
q[['fighterId_1', 'fighterId_2', 'eventDate.date', 'v2_1']]

,fighterId_1,fighterId_2,eventDate.date,v2_1
542,2597,3418,2021-01-16 00:00:01,0.482063
543,240,3398,2021-01-16 00:00:02,0.498381
544,141,453,2021-01-16 00:00:03,0.440628
545,2319,3219,2021-01-16 00:00:04,0.501687
547,3175,3255,2021-01-16 00:00:06,0.386172
...,...,...,...,...
659,430,3587,2021-04-10 00:00:07,0.389904
660,288,3037,2021-04-10 00:00:09,0.558093
661,552,600,2021-04-10 00:00:10,0.381135
662,215,449,2021-04-10 00:00:11,0.548573


In [35]:
q['fighterId_1']

0.48693